In [10]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd

# Split data
df = pd.read_csv('training_data_VT2026.csv')

df['increase_stock'] = df['increase_stock'].map({'high_bike_demand': True, 'low_bike_demand': False}) 
#High demand = True, Low demand = False

train, test = train_test_split(df, test_size=0.2, random_state=1, stratify=df['increase_stock'])
X_train = train.drop(columns = ['increase_stock'])
y_train = train['increase_stock']
X_test = test.drop(columns = ['increase_stock'])
y_test = test['increase_stock']


# Pipeline
pipe = Pipeline([
    ('bagging', BaggingClassifier(estimator=DecisionTreeClassifier(), random_state=1))
])

# Grid
param_grid = {
    'bagging__n_estimators': [10, 20, 50, 100],
    'bagging__max_samples': [0.5, 0.8, 1.0],
    'bagging__estimator__max_depth': [3, 5, None],
    'bagging__estimator__min_samples_split': [2, 5, 10]
}

# Grid search
grid = GridSearchCV(pipe, param_grid, cv=5, scoring='f1', n_jobs=-1)
grid.fit(X_train, y_train)

print("Best params:", grid.best_params_)
print("Best score:", grid.best_score_)

best_model = grid.best_estimator_
y_predict = best_model.predict(X_test)
print("Test accuracy:", accuracy_score(y_test, y_predict))
print(classification_report(y_test, y_predict))


Best params: {'bagging__estimator__max_depth': 5, 'bagging__estimator__min_samples_split': 10, 'bagging__max_samples': 1.0, 'bagging__n_estimators': 50}
Best score: 0.7092485710907963
Test accuracy: 0.896875
              precision    recall  f1-score   support

       False       0.92      0.95      0.94       262
        True       0.76      0.64      0.69        58

    accuracy                           0.90       320
   macro avg       0.84      0.80      0.81       320
weighted avg       0.89      0.90      0.89       320

